In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
path = untar_data(URLs.PETS, dest="/content")
annotations_path = path.joinpath("./annotations/")
images_path = path.joinpath("./images/")

In [ ]:
import os
import glob
import ipywidgets as widgets
from IPython.display import display

images = widgets.Dropdown(options=[os.path.basename(p) for p in images_path.glob("./*.jpg")], description='images:', continuous_update=True)
# trimaps = widgets.Dropdown(options=[os.path.basename(p) for p in annotations_path.glob("./*.png")], description='annotations-trimaps:', continuous_update=True)
# display(images, trimaps)

In [ ]:
out_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = f'Waiting for Result...'
btn_run = widgets.Button(description='Display')
def on_click_classify(change):
  selected_image = images_path.joinpath(images.value)
  img = PILImage.create(selected_image)
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))

btn_run.on_click(on_click_classify)

In [ ]:

VBox([widgets.Label('Do you want to know you pet? Select a Photo...'), images, 
      btn_run, out_pl, lbl_pred])

In [ ]:
import pandas as pd

df = pd.read_csv(annotations_path/'trainval.txt', header=6, sep=' ', names=['image', 'class_id', 'species', 'bread_id'])
df.shape

In [ ]:
# dls = ImageDataLoaders.from_df(df, path=images_path, item_tfms=Resize(500), batch_tfms=aug_transforms(size=250))

def get_x(x): return images_path/(x['image'] + ".jpg")
def get_y(x): return x['class_id']

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(seed=42),
                   get_x=get_x,
                   get_y=get_y, 
                   item_tfms = Resize(500),
                   batch_tfms=aug_transforms(size=250)) 

In [ ]:
dls = dblock.dataloaders(df)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)

In [ ]:
learn.fine_tune(epochs=10, freeze_epochs=2)

In [ ]:
!ls oxford-iiit-pet/images/*.jpg | wc -l


In [ ]:
!cat /content/oxford-iiit-pet/annotations/list.txt | more
